In [1]:
import os
import subprocess
import time
import sys
sys.path.append('/workspace/activation-steering')

In [2]:
# Start Xvfb manually (xvfb-run does this under the hood)
subprocess.Popen([
    'Xvfb', ':99', '-screen', '0', '1280x1024x24', '+extension', 'GLX', '+render', '-noreset'
])

time.sleep(2)  # Give Xvfb time to start

(EE) 
Fatal server error:
(EE) Server is already active for display 99
	If this server is no longer running, remove /tmp/.X99-lock
	and start again.
(EE) 


In [3]:
# Set environment variables to match your working terminal setup
os.environ['DISPLAY'] = ':99'
os.environ['LIBGL_ALWAYS_SOFTWARE'] = '1'
#os.environ['LIBGL_DRIVERS_PATH'] = '/usr/lib/x86_64-linux-gnu/dri' - for google cloud
#os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libffi.so.7'

In [4]:
!glxinfo | grep 'OpenGL renderer string'
!glxinfo | grep 'OpenGL version string'
!glxinfo | grep 'GLX version'
!glxinfo | grep 'renderer'

OpenGL renderer string: llvmpipe (LLVM 15.0.7, 256 bits)
OpenGL version string: 4.5 (Compatibility Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
GLX version: 1.4
    GLX_MESA_copy_sub_buffer, GLX_MESA_query_renderer, GLX_MESA_swap_control, 
    GLX_MESA_query_renderer, GLX_OML_swap_method, GLX_SGIS_multisample, 
Extended renderer info (GLX_MESA_query_renderer):
OpenGL renderer string: llvmpipe (LLVM 15.0.7, 256 bits)


In [5]:
import openai
os.environ["OPENAI_API_KEY"] = "Key Here"
from arguments import get_config
from interfaces import setup_LMP, LMP_interface
from visualizers import ValueMapVisualizer
from envs.rlbench_env import VoxPoserRLBench
from utils import set_lmp_objects
import numpy as np
from rlbench import tasks
from LMP import LMP

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


ModuleNotFoundError: No module named 'rlbench'

## Setup

In [6]:
config = get_config('rlbench')
# uncomment this if you'd like to change the language model (e.g., for faster speed or lower cost)
# for lmp_name, cfg in config['lmp_config']['lmps'].items():
#     cfg['model'] = 'gpt-3.5-turbo'

# initialize env and voxposer ui
visualizer = ValueMapVisualizer(config['visualizer'])
env = VoxPoserRLBench(visualizer=visualizer)
lmps, lmp_env = setup_LMP(env, config, debug=False)
voxposer_ui = lmps['plan_ui']

##################################################
## voxel resolution: [0.0105 0.0131 0.01  ]
##################################################




## Playground

By default we use one of the instructions that come with each task. However, you may treat each task as simply a scene initialization from RLBench, and feel free to try any task you can come up with for the scene.

Note:
- Whether an instruction can be executed or not depends on 1) whether relevant objects are available, and 2) capabilities of the overall algorithm.
- Each execution may produce one or more visualizations. You may view them in "./visualizations/" folder.
- The prompts are adapted with minimal change from the real-world environment in the VoxPoser paper. If a task failure is due to incorrect code generated by the LLM, feel free to modify the relevant prompt in "./prompts/" folder.
- You may view the reward by printing "env.latest_reward". These are computed by RLBench for each task.
- To inspect in viewer without performing any action, you may call "env.rlbench_env._scene.step()" in a loop.

In [7]:
# # uncomment this to show all available tasks in rlbench
# # NOTE: in order to run a new task, you need to add the list of objects (and their corresponding env names) to the "task_object_names.json" file. See README for more details.
# print([task for task in dir(tasks) if task[0].isupper() and not '_' in task])

In [8]:
# below are the tasks that have object names added to the "task_object_names.json" file
# uncomment one to use
# env.load_task(tasks.PutRubbishInBin)
# env.load_task(tasks.LampOff)
# env.load_task(tasks.OpenWineBottle)
# env.load_task(tasks.PushButton)
# env.load_task(tasks.TakeOffWeighingScales)
# env.load_task(tasks.MeatOffGrill)
# env.load_task(tasks.SlideBlockToTarget)
# env.load_task(tasks.TakeLidOffSaucepan)
# env.load_task(tasks.TakeUmbrellaOutOfUmbrellaStand)
env.load_task(tasks.SetTheTable)

descriptions, obs = env.reset()
set_lmp_objects(lmps, env.get_object_names())  # set the object names to be used by voxposer

Loaded task: set_the_table


In [9]:
instruction = np.random.choice(descriptions)
voxposer_ui(instruction)

(using cache) *** OpenAI API call took 0.00s ***
########################################
## "planner" generated code
## context: "objects = ['chocolate', 'apple', 'granola', 'soda']"
########################################
objects = ['chocolate', 'apple', 'granola', 'soda']
# Query: Pick up a snack for me.
composer("grasp the chocolate")
composer("back to default pose")
# done


(using cache) *** OpenAI API call took 0.00s ***
########################################
## "composer" generated code
########################################
# Query: grasp the chocolate.
movable = parse_query_obj('gripper')
affordance_map = get_affordance_map('a point at the center of the chocolate')
gripper_map = get_gripper_map('open everywhere except 1cm around the chocolate')
execute(movable, affordance_map=affordance_map, gripper_map=gripper_map)


(using cache) *** OpenAI API call took 0.00s ***
########################################
## "parse_query_obj" generated code
## context: "objects = ['choc

*** OpenAI API call took 3.03s ***
########################################
## "get_gripper_map" generated code
########################################
# Query: open everywhere except 1cm around the chocolate.
gripper_map = get_empty_gripper_map()
# open everywhere
gripper_map[:, :, :] = 1
# close when 1cm around the chocolate
chocolate = parse_query_obj('chocolate')
set_voxel_by_radius(gripper_map, chocolate.position, radius_cm=1, value=0)
ret_val = gripper_map


*** OpenAI API call took 1.62s ***
########################################
## "parse_query_obj" generated code
## context: "objects = ['chocolate', 'apple', 'granola', 'soda']"
########################################
objects = ['chocolate', 'apple', 'granola', 'soda']
# Query: chocolate.
chocolate = detect('chocolate')
ret_val = chocolate


(using cache) *** OpenAI API call took 0.00s ***
########################################
## "parse_query_obj" generated code
## context: "objects = ['chocolate', 'apple', 'granola', 's